In [1]:
import numpy as np
import pandas as pd

In [2]:
def read(path):
    df=pd.read_csv(path,delimiter='\t',header=None)
    df.rename(columns={0:'image_id',1:'x',2:'y',3:'width',4:'height',5:'english',6:'hindi'},inplace=True)
    df.drop(columns=['x','y','width','height'],axis=1,inplace=True)
    return df

In [3]:

train_path=r'https://raw.githubusercontent.com/Sk4467/datasets/main/hindi-visual-genome-train.txt?token=GHSAT0AAAAAACCK2RSGTDICOGIES45XS2QQZE5ZYJA'
test_path=r'https://raw.githubusercontent.com/Sk4467/datasets/main/hindi-visual-genome-dev.txt'

In [4]:
train=read(train_path)
test=read(test_path)

In [38]:
# !pip install datasets transformers evaluate sacrebleu

###########################################################################
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [39]:
import datasets
from datasets import Dataset, DatasetDict
train_dataset = datasets.Dataset.from_pandas(train)
train_data = datasets.DatasetDict({'train':train_dataset})

In [40]:
test_dataset = datasets.Dataset.from_pandas(test)
test_data = datasets.DatasetDict({'test':test_dataset})

In [41]:
from transformers import AutoTokenizer
checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [68]:
# prefix="translate English to hindi : "
def preprocess(data):

    inputs=[text for text in data['english']]
    target=[text for text in data['hindi']]

    model_inputs = {
        'id': data['image_id'] ,
        'translation': target,  # assign 'translation' directly to 'target'
        'input_ids': [],
        'attention_mask': [],
        'labels' : []
    }

    try:
        # Tokenize all inputs and targets at once, which is more efficient
        tokenized_samples = tokenizer(inputs, max_length=128, truncation=True)
        labels= tokenizer(target,max_length=128,truncation=True)

        # Assign the tokenized samples directly to 'input_ids', 'attention_mask', and 'labels'
        model_inputs['input_ids'] = tokenized_samples['input_ids']
        model_inputs['attention_mask'] = tokenized_samples['attention_mask']
        model_inputs['labels'] = labels['input_ids']  
        
    except Exception as e:
        print(f"Error occurred: {e}")

    return model_inputs

In [69]:
tokenized_train_data=train_data.map(preprocess,batched=True)
tokenized_test_data=test_data.map(preprocess,batched=True)


Map:   0%|          | 0/28930 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [70]:
tokenized_train_data['train']['labels']

[[256047, 4228, 1224, 15695, 248586, 12972, 68360, 654, 2],
 [256047, 102597, 114755, 114698, 2],
 [256047, 51865, 483, 69301, 52135, 2207, 2],
 [256047,
  1224,
  242641,
  1550,
  11430,
  248257,
  1035,
  41223,
  237383,
  248277,
  248173,
  100212,
  2],
 [256047, 879, 2087, 4085, 483, 16515, 246836, 879, 1224, 47148, 654, 2],
 [256047, 1224, 67442, 248173, 103815, 2032, 1035, 49667, 2],
 [256047, 96295, 231607, 41857, 42021, 2],
 [256047, 182537, 193017, 46660, 1550, 654, 2],
 [256047, 182537, 1186, 12302, 45317, 2207, 2],
 [256047, 141919, 879, 80502, 707, 10037, 16642, 2],
 [256047, 4228, 1224, 95409, 2648, 248397, 150055, 654, 2],
 [256047, 23315, 231607, 1550, 2649, 27126, 56231, 2],
 [256047, 45123, 9172, 1224, 3610, 248290, 1186, 654, 2],
 [256047, 1224, 193017, 219697, 3479, 248217, 1996, 8247, 2],
 [256047,
  51310,
  1561,
  1213,
  483,
  39535,
  1160,
  193017,
  45261,
  76520,
  10800,
  111400,
  246512,
  2],
 [256047, 1279, 62185, 483, 4576, 112771, 17647, 1164

In [71]:
test_ids = tokenized_train_data['train']['labels']

In [72]:
predictions = [tokenizer.decode(ids, skip_special_tokens=True) for ids in test_ids]

In [73]:
predictions

['यह एक इनडोर दृश्य है',
 'कंप्यूटर स्क्रीन चालू',
 'आदमी के छोटे बाल हैं',
 'एक वयस्क की गोद में फोटो एल्बम खुला',
 'काली कार के पास लड़कियों का एक समूह है',
 'एक घुमक्कड़ में बच्चा',
 'लंबा धातु प्रकाश पोस्ट',
 'दीवार सफेद रंग की है',
 'दीवार पर कई चित्र हैं',
 'सागर का सामना कर रही महिला',
 'यह एक कार्यालय लेआउट है',
 'चार धातु की कुर्सियाँ',
 'क्लटर एक मेज पर है',
 'एक सफेद माइक्रोवेव ओवन',
 'चौराहे के माध्यम से सफेद एसयूवी ड्राइविंग',
 'छतरी के साथ सड़क पार करते व्यक्ति',
 'इमारत पर झुकाव में आदमी ग्रे पैंट में',
 'भवन पर खिड़की',
 'कारों के बीच में खड़ा एक आदमी',
 'दीवार पर चित्रकला',
 'कुर्सी का पैर',
 'एक नाव की खिड़की',
 'कैबिनेट लकड़ी है',
 'एक आदमी एक किताब पढ़ रहा है',
 'पानी में तैरता हुआ व्यक्ति',
 'चमकती छाया के साथ दीपक',
 'प्लेट पर फल',
 'एक चीनी मिट्टी के बरतन सफ़ेद प्लेट',
 'काला खेल नियंत्रक',
 'कार की डिक्की में कुछ डालने वाला व्यक्ति।',
 'एक सड़क के साथ ग्रीन कार पार्क किया',
 'गली पर कार',
 'निर्माण कर्मचारी कवरलेस पहनते हैं',
 'पीले दरवाजे के साथ एक काली कार',
 

In [75]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='facebook/nllb-200-distilled-600M')

In [76]:
import evaluate

metric = evaluate.load("sacrebleu")

In [77]:
import numpy as np
def postprocess(preds,labels):
    preds=[pred.strip() for pred in preds]
    labels=[[label.strip()] for label in labels]
    return preds,labels


def compute_metrics(eval_preds):
    preds,labels=eval_preds
    if isinstance(preds,tuple):
        preds=preds[0]
    decoded_preds=tokenizer.batch_decode(preds,skip_special_tokens=True)
        
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels,skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [78]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained('facebook/nllb-200-distilled-600M')

In [79]:
training_args = Seq2SeqTrainingArguments(
    output_dir="fine_tune_model",
    # report_to="wandb",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    eval_accumulation_steps=50,
    gradient_accumulation_steps=4,
    predict_with_generate=True,
    # push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data['train'],
    eval_dataset=tokenized_test_data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [80]:
# trainer.train()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: iftesha1. Use `wandb login --relogin` to force relogin


You're using a NllbTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
0,0.598800,0.629158,48.708500,10.318600
2,0.518200,0.612843,49.860500,10.301600
2,0.458300,0.608587,49.587500,10.316600
4,0.420300,0.602445,49.447500,10.322600
4,0.397800,0.606285,49.491100,10.325700


TrainOutput(global_step=6025, training_loss=0.4926386732679185, metrics={'train_runtime': 1499.5382, 'train_samples_per_second': 96.463, 'train_steps_per_second': 4.018, 'total_flos': 3338568548843520.0, 'train_loss': 0.4926386732679185, 'epoch': 5.0})

In [81]:
# trainer.evaluate()

{'eval_loss': 0.606285035610199,
 'eval_bleu': 49.4911,
 'eval_gen_len': 10.3257,
 'eval_runtime': 10.1683,
 'eval_samples_per_second': 98.148,
 'eval_steps_per_second': 16.424,
 'epoch': 5.0}

In [83]:
# trainer.save_model('/home/jupyter/notebooks/notebook/fine-tune/')

In [5]:
import torch
torch.cuda.empty_cache()

In [27]:
path=r'https://raw.githubusercontent.com/Sk4467/datasets/main/hindi-visual-genome-test.txt'
pred=read(path)

In [28]:
pred

,image_id,english,hindi
0,780,the orange colored traffic cone,नारंगी रंग यातायात शंकु
1,2376980,A stop light,एक स्टॉप लाइट
2,2337791,three zebras in the wild,जंगल में तीन जेब्रा
3,2339,cars parked along the side of the road,सड़क के किनारे खड़ी कारें
4,2347468,a man climbing into a train,एक व्यक्ति रेलगाड़ी में चढ़ रहा है
...,...,...,...
1590,2415214,a stoplight lit to green,हरी दुर्दशा को प्रकाशित
1591,2354097,surfboards stacked on beach,समुद्र तट पर स्टैक किया गया
1592,2357303,a white red and black bus,एक सफेद लाल और काली बस
1593,2410308,green broccoli with chicken,मुर्गा के साथ हरी ब्रोकोली


In [29]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.translate.bleu_score import corpus_bleu

model_name = '/home/jupyter/notebooks/notebook/fine-tune/'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")


In [30]:
test_sentences=pred['english'].to_list()


In [31]:
test_sentences

['the orange colored traffic cone',
 'A stop light',
 'three zebras in the wild',
 'cars parked along the side of the road',
 'a man climbing into a train',
 'This is a person',
 'Box of small oranges in a box.',
 'Signs about TRAMS on a post.',
 'a white box filled with doughnuts',
 'person wearing a black hat',
 'the bird is black',
 'a girl is standing.',
 'window on a building',
 'a Southwest airliner seen in the distance between two tall buildings',
 'the hawk on the branch',
 'a table full of vegetables',
 'face of the cow',
 'Man wearing military clothes',
 'A glass of wine',
 'a person walking on a sidewalk',
 'this is a frisbee',
 'Colorful kite',
 'headlights on the bus',
 'a dog jumpin up',
 'the girl is reading',
 'a tower surround by water',
 'a girl playing tennis',
 'the giraffe is extending his neck',
 'a red bench on the sidewalk',
 'A large plate to the front',
 'red light house with a black roof',
 'a water glass on a table',
 'headlight',
 'red car on the street',
 

In [156]:
enc_inp=tokenizer(test_sentences[4],return_tensors='pt').to(torch.device('cuda'))
enc_inp
with torch.no_grad():
    output=model.generate(
    input_ids=enc_inp['input_ids'],
    attention_mask=enc_inp['attention_mask'],
    max_length=128
    )
output
pred_sent=tokenizer.batch_decode(output,skip_special_tokens=True)[0]
act_sent=pred['hindi'][4]
pred_sent
act_sent
actual_tokenized =nltk.word_tokenize(act_sent)
predicted_tokenized=nltk.word_tokenize(pred_sent)
actual_tokenized

['सूक्ष्म', 'पतले', 'लाल', 'बाल']

In [32]:
#encoding the inputs test sentences
encoded_inputs=tokenizer(test_sentences,padding=True,truncation=True,return_tensors='pt').to(torch.device("cuda"))


In [33]:
encoded_inputs

{'input_ids': tensor([[256047,    349,    618,  ...,      1,      1,      1],
        [256047,     70,  18067,  ...,      1,      1,      1],
        [256047,  29213, 106408,  ...,      1,      1,      1],
        ...,
        [256047,      9,  84869,  ...,      1,      1,      1],
        [256047,  93716,   8610,  ...,      1,      1,      1],
        [256047,      9, 183598,  ...,      1,      1,      1]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [34]:
with torch.no_grad():
    outputs=model.generate(
    input_ids=encoded_inputs['input_ids'],
    attention_mask=encoded_inputs['attention_mask'],
    max_length=128
    )

In [35]:
outputs

tensor([[     2, 256047, 163094,  ...,      1,      1,      1],
        [     2, 256047,   1224,  ...,      1,      1,      1],
        [     2, 256047, 172247,  ...,      1,      1,      1],
        ...,
        [     2, 256047,   1224,  ...,      1,      1,      1],
        [     2, 256047,  65825,  ...,      1,      1,      1],
        [     2, 256047,   1224,  ...,      1,      1,      1]],
       device='cuda:0')

In [36]:
translated_sentences=tokenizer.batch_decode(outputs,skip_special_tokens=True)

In [37]:
translated_sentences

['नारंगी रंग का यातायात शंकु',
 'एक स्टॉप लाइट',
 'जंगली में तीन ज़ेबरा',
 'सड़क के किनारे खड़ी कारें',
 'एक आदमी एक ट्रेन में चढ़ रहा है',
 'यह एक व्यक्ति है',
 'एक बॉक्स में छोटे संतरे का डिब्बा।',
 'एक पोस्ट पर ट्राम के बारे में संकेत।',
 'डोनट्स से भरा एक सफेद बॉक्स',
 'काली टोपी पहने व्यक्ति',
 'पक्षी काला है',
 'एक लड़की खड़ी है।',
 'एक इमारत पर खिड़की',
 'एक दक्षिण पश्चिम हवाई जहाज दो ऊंची इमारतों के बीच की दूरी पर देखा गया',
 'शाखा पर बाज',
 'सब्जियों से भरी मेज',
 'गाय का चेहरा',
 'सैन्य कपड़े पहने आदमी',
 'शराब का एक गिलास',
 'फुटपाथ पर चलने वाला व्यक्ति',
 'यह एक फ्रिसबी है',
 'रंगीन पतंग',
 'बस में हेडलाइट्स',
 'एक कुत्ता कूद गया',
 'लड़की पढ़ रही है',
 'एक टावर पानी से घिरा हुआ है',
 'एक लड़की टेनिस खेल रही है',
 'जिराफ उसकी गर्दन बढ़ा रहा है',
 'फुटपाथ पर एक लाल बेंच',
 'सामने की एक बड़ी प्लेट',
 'एक काले छत के साथ लाल प्रकाश घर',
 'एक मेज पर पानी का गिलास',
 'हेडलाइट',
 'सड़क पर लाल कार',
 'तीन हाथी हैं',
 'पेड़ के हरे पत्ते होते हैं',
 'दो लोग पर्वतारोहण करते हैं',
 'का

In [38]:
def write_to_file(sentences,path):
    with open(path,'w',encoding='utf-8') as file:
        for sentence in sentences:
            file.write(sentence+'\n')
            
            


In [44]:
path='hindi_translation_eval_test.txt'

In [45]:
write_to_file(translated_sentences,path)

In [39]:
import nltk
from nltk.translate.bleu_score import corpus_bleu,sentence_bleu

In [40]:
actual_translations=pred['hindi'].to_list()
predicted_translations=translated_sentences

In [41]:
len(predicted_translations)

1595

In [42]:
actual_tokenized=[[nltk.word_tokenize(sentence)] for sentence in actual_translations]
predicted_tokenized=[nltk.word_tokenize(sentence) for sentence in predicted_translations]

In [43]:

bleu_score=corpus_bleu(actual_tokenized,predicted_tokenized,weights=(1,0,0,0))
print(f"BLEU-score: {bleu_score}")

BLEU-score: 0.7044754037726544
